In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

import data_collection

Here I will try to get the spotify features that are currently missing in the initial dataset

In [4]:
CLIENT_ID = 'x'
CLIENT_SECRET = 'x'
REDIRECT_URL = 'http://localhost:5000'

In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
#Read this before any data is filled and the new file is created
#data = pd.read_csv("hot_100_spotify.csv")

data = pd.read_csv("filled_spotify.csv")

On the first run, I created a file with all rows that have a missing spotify feature in them. On consecutive runs I will use the created file, fill some values, update the **filled_spotify** file and delete them from the created file, so that I can thin it out little by little.

In [8]:
#Ran this the first time the file with missing values was created
#missing = data[data.isnull().any(axis = 1)]
#missing_data = missing.iloc[:, 6:]
#missing_data['song'] = missing.song
#missing_data['index_empty_val'] = missing['Unnamed: 0']
#missing_data.to_csv("missing_spotify_features.csv")

missing_data = pd.read_csv("missing_spotify_features.csv")

In [9]:
missing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6180 entries, 0 to 6179
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          6180 non-null   int64  
 1   spotify_genre       3666 non-null   object 
 2   track_duration_s    196 non-null    float64
 3   explicit_track      196 non-null    object 
 4   danceability        142 non-null    float64
 5   energy              142 non-null    float64
 6   key                 142 non-null    float64
 7   loudness            142 non-null    float64
 8   mode                142 non-null    float64
 9   speechiness         142 non-null    float64
 10  acousticness        142 non-null    float64
 11  instrumentalness    142 non-null    float64
 12  liveness            142 non-null    float64
 13  valence             142 non-null    float64
 14  tempo               142 non-null    float64
 15  spotify_popularity  196 non-null    float64
 16  main_a

In [10]:
filled_data = pd.DataFrame(columns = missing_data.columns)

In [11]:
for i in range(20):
    if (len(missing_data) >= 20):
        current_batch = missing_data.head(20)
        spotify_features_list = []
        for _, row in current_batch.iterrows():
            missing_row_index = row['index_empty_val']
            artist = row['main_artist']
            song = row['song']
            spotify_info = data_collection.get_track_details_by_name(missing_row_index, song, artist, sp)
            spotify_features_list.append(spotify_info)
        no_nulls = [i for i in spotify_features_list if i != None] #Get only non-null values
        filled_batch = pd.DataFrame(no_nulls) #A dataframe from the gathered infomration
        filled_data = pd.concat([filled_data, filled_batch], ignore_index = True)

        missing_data = missing_data.reset_index(drop = True).drop(range(20))

C:\Users\ferad\AppData\Local\Temp\ipykernel_11984\1703369018.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  filled_data = pd.concat([filled_data, filled_batch], ignore_index = True)


In [12]:
missing_data.drop(columns = ['Unnamed: 0']).to_csv("missing_spotify_features.csv") #Overwrite the file, should now be smaller
filled_data.to_csv("filled_data.csv") #Export the last filled data to a csv just in case

In [13]:
filling = pd.merge(left = data, right = filled_data, left_index = True, right_on = "index_empty_val", how = 'left')
column_list = [
    'spotify_genre', 'track_duration_s', 'explicit_track',
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness', 'liveness',
    'valence', 'tempo', 'spotify_popularity'
]
for col in column_list:
    col_x = col + "_x"
    col_y = col + "_y"
    filling[col_x] = filling[col_x].fillna(filling[col_y]) #Fill the original columns with the information from the merged ones

In [14]:
filling = filling.reset_index(drop = True)
filling = filling.iloc[:, 1:23] #Keep only the '_x' columns
for col in filling.columns:
    filling = filling.rename(columns = {col : col.replace("_x", "")}) #Clean the names

In [15]:
filling.to_csv("filled_spotify.csv")

In [16]:
filling.columns

Index(['song', 'artist', 'peak_position', 'weeks_on_chart', 'date',
       'spotify_genre', 'track_duration_s', 'explicit_track', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'spotify_popularity', 'main_artist', 'artist_list'],
      dtype='object')

In [17]:
filling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31447 entries, 0 to 31446
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   song                31447 non-null  object 
 1   artist              31447 non-null  object 
 2   peak_position       31447 non-null  int64  
 3   weeks_on_chart      31447 non-null  int64  
 4   date                31447 non-null  object 
 5   spotify_genre       29247 non-null  object 
 6   track_duration_s    25776 non-null  float64
 7   explicit_track      25776 non-null  object 
 8   danceability        25722 non-null  float64
 9   energy              25722 non-null  float64
 10  key                 25722 non-null  float64
 11  loudness            25722 non-null  float64
 12  mode                25722 non-null  float64
 13  speechiness         25722 non-null  float64
 14  acousticness        25722 non-null  float64
 15  instrumentalness    25722 non-null  float64
 16  live

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31447 entries, 0 to 31446
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          31447 non-null  int64  
 1   song                31447 non-null  object 
 2   artist              31447 non-null  object 
 3   peak_position       31447 non-null  int64  
 4   weeks_on_chart      31447 non-null  int64  
 5   date                31447 non-null  object 
 6   spotify_genre       28857 non-null  object 
 7   track_duration_s    25387 non-null  float64
 8   explicit_track      25387 non-null  object 
 9   danceability        25333 non-null  float64
 10  energy              25333 non-null  float64
 11  key                 25333 non-null  float64
 12  loudness            25333 non-null  float64
 13  mode                25333 non-null  float64
 14  speechiness         25333 non-null  float64
 15  acousticness        25333 non-null  float64
 16  inst